<a href="https://colab.research.google.com/github/cminy/FC2204/blob/main/sql_001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [156]:
# mysql 사용을 위한 패키지 설치
!sudo apt-get install libmysqlclient-dev
!pip install mysqlclient

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libmysqlclient-dev is already the newest version (5.7.37-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [157]:
# 패키지 코드를 사용할 수 있도록 패키지 import
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import MySQLdb

In [158]:
db = MySQLdb.connect(host="2.tcp.ngrok.io", port=19683, user="root", passwd="mk246900", auth_plugin='mysql_native_password', db="world", charset="utf8")
curs = db.cursor()

In [159]:
# --------- 6p
# world 데이터 베이스에서 각 국가의 국가코드, 국가이름, 인구밀도(1m2당 인구수)를 출력하는 쿼리를 작성하세요.
qr01 = 'SHOW TABLES;'
pd.read_sql(qr01, db)


,Tables_in_world
0,city
1,country
2,countrylanguage


In [160]:
qr01 = 'DESC country;' #DDL
pd.read_sql(qr01, db)

,Field,Type,Null,Key,Default,Extra
0,Code,char(3),NO,PRI,,
1,Name,char(52),NO,,,
2,Continent,"enum('Asia','Europe','North America','Africa',...",NO,,Asia,
3,Region,char(26),NO,,,
4,SurfaceArea,"decimal(10,2)",NO,,0.00,
5,IndepYear,smallint,YES,,None,
6,Population,int,NO,,0,
7,LifeExpectancy,"decimal(3,1)",YES,,None,
8,GNP,"decimal(10,2)",YES,,None,
9,GNPOld,"decimal(10,2)",YES,,None,


In [161]:
qr01 = 'select*from country;'
test = pd.read_sql(qr01, db)
test.head(3)

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ABW,Aruba,North America,Caribbean,193.0,NaN,103000,78.4,828.0,793.0,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129.0,AW
1,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF
2,AGO,Angola,Africa,Central Africa,1246700.0,1975.0,12878000,38.3,6648.0,7984.0,Angola,Republic,José Eduardo dos Santos,56.0,AO


In [162]:
qr01 = 'select code, name, population/surfacearea as "인구밀도" from country;'
an01 = pd.read_sql(qr01, db)
an01.head(3)

,code,name,인구밀도
0,ABW,Aruba,533.6788
1,AFG,Afghanistan,34.8418
2,AGO,Angola,10.3297


In [163]:
# world 데이터 베이스에서 각 국가의 국가코드, 국가이름, 1인당 GNP를 출력하는 쿼리를 작성
qr02 = '''select code, name, 
gnp/population as "1인당GNP(GPP)" 
from country;'''
an02 = pd.read_sql(qr02, db)
an02.head(3)

,code,name,1인당GNP(GPP)
0,ABW,Aruba,0.008039
1,AFG,Afghanistan,0.000263
2,AGO,Angola,0.000516


In [164]:
# ------ 7p
# world 데이터 베이스에서 국가코드, 국가이름, 대륙, asia 대륙이면 1(True)을 출력하는 쿼리
qr03 = """
  select case when continent = 'Asia' then 1 end as '아시아'
    , code, name, continent
  from country;"""
an03 = pd.read_sql(qr03, db)
an03.head()

,아시아,code,name,continent
0,NaN,ABW,Aruba,North America
1,1.0,AFG,Afghanistan,Asia
2,NaN,AGO,Angola,Africa
3,NaN,AIA,Anguilla,North America
4,NaN,ALB,Albania,Europe


In [165]:
qr03 = """select continent = 'Asia'as is_asia, code, name, continent from country;"""
an03 = pd.read_sql(qr03, db)
an03.head()

,is_asia,code,name,continent
0,0,ABW,Aruba,North America
1,1,AFG,Afghanistan,Asia
2,0,AGO,Angola,Africa
3,0,AIA,Anguilla,North America
4,0,ALB,Albania,Europe


In [166]:
# world 데이터 베이스에서 국가코드, 국가이름, 독립년도, 독립년도가 1900년 이후에 독립했으면 1(True)을 출력하는 쿼리를 작성하세요.
qr04 = "select code, name, indepyear, case when indepyear >= 1900 then 1 end as '1900년 이후' from country;"
an04 = pd.read_sql(qr04, db)
an04.head()

,code,name,indepyear,1900년 이후
0,ABW,Aruba,NaN,NaN
1,AFG,Afghanistan,1919.0,1.0
2,AGO,Angola,1975.0,1.0
3,AIA,Anguilla,NaN,NaN
4,ALB,Albania,1912.0,1.0


In [167]:
qr04 = "select code, name, indepyear, indepyear >= 1900 as '1900년 이후' from country;"
an04 = pd.read_sql(qr04, db)
an04.head()

,code,name,indepyear,1900년 이후
0,ABW,Aruba,NaN,NaN
1,AFG,Afghanistan,1919.0,1.0
2,AGO,Angola,1975.0,1.0
3,AIA,Anguilla,NaN,NaN
4,ALB,Albania,1912.0,1.0


In [168]:
# world 데이터 베이스에서 국가코드, 국가이름, 인구수, 기대수명, 인구수가 5천만 이상이고, 기대수명이 70세 이상이면 1(True)을 출력하는 쿼리를 작성하세요.
qr05 = "select code, name, population, lifeexpectancy, case when (population >= 5000*10000) AND (lifeexpectancy >= 70) then 1 end as 'TRUE' from country;"
an05 = pd.read_sql(qr05, db)
an05.head(3)

,code,name,population,lifeexpectancy,TRUE
0,ABW,Aruba,103000,78.4,NaN
1,AFG,Afghanistan,22720000,45.9,NaN
2,AGO,Angola,12878000,38.3,NaN


In [169]:
an05.sort_values('TRUE')

,code,name,population,lifeexpectancy,TRUE
41,CHN,China,1277558000,71.4,1.0
56,DEU,Germany,82164700,77.4,1.0
72,FRA,France,59225700,78.8,1.0
76,GBR,United Kingdom,59623400,77.7,1.0
106,ITA,Italy,57680000,79.0,1.0
...,...,...,...,...,...
234,YEM,Yemen,18112000,59.8,NaN
235,YUG,Yugoslavia,10640000,72.4,NaN
236,ZAF,South Africa,40377000,51.1,NaN
237,ZMB,Zambia,9169000,37.2,NaN


In [170]:
# world 데이터 베이스에서 국가코드, 국가이름, 인구수, 기대수명, 인구수가 5천만 이상이고, 기대수명이 70세 이상이면 1(True)을 출력하는 쿼리를 작성하세요.
qr05 = "select code, name, population, lifeexpectancy, (population >= 5000*10000) AND (lifeexpectancy >= 70) as 'TRUE' from country;"
an05 = pd.read_sql(qr05, db)
an05.head(3)

,code,name,population,lifeexpectancy,TRUE
0,ABW,Aruba,103000,78.4,0
1,AFG,Afghanistan,22720000,45.9,0
2,AGO,Angola,12878000,38.3,0


In [171]:
#------ 9, 10
qr = "select * from country where continent in ('Asia', 'Africa');"
an = pd.read_sql(qr, db)
an.head(3)

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF
1,AGO,Angola,Africa,Central Africa,1246700.0,1975.0,12878000,38.3,6648.0,7984.0,Angola,Republic,José Eduardo dos Santos,56.0,AO
2,ARE,United Arab Emirates,Asia,Middle East,83600.0,1971.0,2441000,74.1,37966.0,36846.0,Al-Imarat al-´Arabiya al-Muttahida,Emirate Federation,Zayid bin Sultan al-Nahayan,65.0,AE


In [172]:
qr = 'select * from country where code like "Z%";'
an = pd.read_sql(qr, db)
an.head(3)

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ZAF,South Africa,Africa,Southern Africa,1221037.0,1910,40377000,51.1,116729.0,129092.0,South Africa,Republic,Thabo Mbeki,716,ZA
1,ZMB,Zambia,Africa,Eastern Africa,752618.0,1964,9169000,37.2,3377.0,3922.0,Zambia,Republic,Frederick Chiluba,3162,ZM
2,ZWE,Zimbabwe,Africa,Eastern Africa,390757.0,1980,11669000,37.8,5951.0,8670.0,Zimbabwe,Republic,Robert G. Mugabe,4068,ZW


In [173]:
# africa 대륙에서 인구수가 2천만이상인 국가출력
qr = '''select code, name, population from country
where (continent = 'Africa') and (population >= 2000*10000);
'''
an = pd.read_sql(qr, db)
an.head(3)
an.to_csv("datatest.csv", index = False, encoding='utf-8-sig')

,code,name,population
0,COD,"Congo, The Democratic Republic of the",51654000
1,DZA,Algeria,31471000
2,EGY,Egypt,68470000


In [174]:
qr = '''select code, name, population from country
where population between 2000*10000 and 5000*10000;
'''
an = pd.read_sql(qr, db)
an.head(3)

,code,name,population
0,AFG,Afghanistan,22720000
1,ARG,Argentina,37032000
2,CAN,Canada,31147000


In [175]:
# ------ 11
# 국가 코드를 알파벳 순으로 정렬하고 같은 국가 코드를 가지면 인구순으로 내림차순으로 정렬
qr = 'select * from city order by countrycode ASC, population DESC;'
an = pd.read_sql(qr, db)
an.head(10)

,ID,Name,CountryCode,District,Population
0,129,Oranjestad,ABW,–,29034
1,1,Kabul,AFG,Kabol,1780000
2,2,Qandahar,AFG,Qandahar,237500
3,3,Herat,AFG,Herat,186800
4,4,Mazar-e-Sharif,AFG,Balkh,127800
5,56,Luanda,AGO,Luanda,2022000
6,57,Huambo,AGO,Huambo,163100
7,58,Lobito,AGO,Benguela,130000
8,59,Benguela,AGO,Benguela,128300
9,60,Namibe,AGO,Namibe,118200


In [176]:
# 인구가 많은 상위 6위 ~ 8위의 3개 국가 데이터를 출력
qr = 'select * from country order by population DESC limit 3 offset 5;'
an = pd.read_sql(qr, db)
an.head()

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,PAK,Pakistan,Asia,Southern and Central Asia,796095.0,1947,156483000,61.1,61289.0,58549.0,Pakistan,Republic,Mohammad Rafiq Tarar,2831,PK
1,RUS,Russian Federation,Europe,Eastern Europe,17075400.0,1991,146934000,67.2,276608.0,442989.0,Rossija,Federal Republic,Vladimir Putin,3580,RU
2,BGD,Bangladesh,Asia,Southern and Central Asia,143998.0,1971,129155000,60.2,32852.0,31966.0,Bangladesh,Republic,Shahabuddin Ahmad,150,BD


In [ ]:
# --- 마무리 Quiz
# Q1 : 한국의 도시 중에서 인구가 100만이 넘는 도시를 인구수 순으로 내림차순 출력 (국가코드, 도시이름, 도시 인구수)
qr = 'DESC city;'
pd.read_sql(qr, db)

In [ ]:
qr = '''
select countrycode, name, population
from city
where (countrycode = 'KOR') AND (population >= 100*10000)
order by population DESC; 
'''
an = pd.read_sql(qr, db)
an.head()

In [ ]:
# Q2 : 1940년 ~ 1950년 사이에 독립한 국가들 중에 GNP가 10만이 넘는 국가를 GNP 내림차순 출력 (국가코드, 국가이름, 대륙, GNP)
qr = '''
select code, name, continent, GNP, indepyear
from country
where (indepyear between 1940 and 1950) AND (GNP >= 10*10000)
order by gnp DESC;
'''
an = pd.read_sql(qr, db)
an

In [ ]:
# Q3 : 영화설명에 drama가 포함되고 관람등급이 R인 영화에서 상영시간이 가장 긴 5개의 영화 내림차순으로 출력
# film_id, title, description, rating, length
qr = 'USE sakila;'
curs.execute(qr)
qr = 'select database();'
an = pd.read_sql(qr, db)
an

In [ ]:
qr = 'select * from film;'
an = pd.read_sql(qr, db)
an.head(1)

In [ ]:
qr = '''
    select film_id, title, description, rating, length
    from film
    where (description like "%drama%") AND (rating = 'R')
    order by length DESC
    limit 5;
'''
an = pd.read_sql(qr, db)
an

In [177]:
# FIN.